In [85]:
import pandas as pd
#导入数据
data = pd.read_csv('custom.csv')

In [86]:
data=data[['user_id','c_time','colt','pay']]
data.columns=['id','R','F','M']

In [87]:
data.head()

,id,R,F,M
0,211918,98,8,398.0
1,212581,1,19,1086.0
2,457900,14,182,2752.0
3,533683,51,367,2376.0
4,550530,255,18,298.0


# 定义分值函数

In [33]:
def Recency(x):
    if x>=0 and x<5:
        return 5
    elif x>=5 and x<15:
        return 4
    elif x>=15 and x<30:
        return 3
    elif x>=30 and x<90:
        return 2
    elif x>=90:
        return 1
    
def Frequency(x):
    if x>=0 and x<10:
        return 1
    elif x>=10 and x<30:
        return 2
    elif x>=30 and x<50:
        return 3
    elif x>=50 and x<100:
        return 4
    elif x>=100:
        return 5
    
def Monetory(x):
    if x>=0 and x<300:
        return 1
    elif x>=300 and x<1000:
        return 2
    elif x>=1000 and x<3000:
        return 3
    elif x>=3000 and x<10000:
        return 4
    elif x>=1000:
        return 5
    

In [88]:
#分值转化
data["R-SCORE"] = data["R"].apply(Recency)
data["F-SCORE"] = data["F"].apply(Frequency)
data["M-SCORE"] = data["M"].apply(Monetory)

In [89]:
#查看分组情况
grouped_r=data["R-SCORE"].groupby(data["R-SCORE"]).count()
grouped_f=data["F-SCORE"].groupby(data["F-SCORE"]).count()
grouped_m=data["M-SCORE"].groupby(data["M-SCORE"]).count()

In [90]:
grouped_r,grouped_f,grouped_m

(R-SCORE
 1    8777
 2    8969
 3    6211
 4    8418
 5    8397
 Name: R-SCORE, dtype: int64, F-SCORE
 1     6098
 2    17045
 3     6993
 4     5896
 5     4740
 Name: F-SCORE, dtype: int64, M-SCORE
 1     1930
 2    23920
 3    12648
 4     2176
 5       98
 Name: M-SCORE, dtype: int64)

In [93]:
#查看平均值
avg_r = data["R-SCORE"].mean()
avg_f = data["F-SCORE"].mean()
avg_m = data["M-SCORE"].mean()
display(avg_r,avg_f,avg_m)

2.967845580300206

2.659938192877465

2.376827234376533

In [94]:
#数据预览
 
import pandas as pd
import pandas_profiling
pandas_profiling.ProfileReport(data)

Number of variables,7
Number of observations,40772
Total Missing (%),0.0%
Total size in memory,2.2 MiB
Average record size in memory,56.0 B
Numeric,7
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [95]:
#判断函数
def judge(x):
    if x>avg:
        return 1
    else:
        return 0

In [96]:
#0-1分类
for avg in [avg_r,avg_f,avg_m]:
    if avg==avg_r:
        data["R-SCORE是否大于均值"] = data["R-SCORE"].apply(judge)
    elif avg==avg_f:
        data["F-SCORE是否大于均值"] = data["F-SCORE"].apply(judge)
    elif avg==avg_m:
        data["M-SCORE是否大于均值"] = data["M-SCORE"].apply(judge)

In [97]:
#定义RFM函数
def RFM(x):
    if x.iloc[0]==1 and x.iloc[1]==1 and x.iloc[2]==1:
        return "重要价值客户"
    elif x.iloc[0]==1 and x.iloc[1]==1 and x.iloc[2]==0:
        return "重要潜力客户"
    elif x.iloc[0]==1 and x.iloc[1]==0 and x.iloc[2]==1:
        return "重要发展客户"
    elif x.iloc[0]==1 and x.iloc[1]==0 and x.iloc[2]==0:
        return "新客户"
    elif x.iloc[0]==0 and x.iloc[1]==1 and x.iloc[2]==1:
        return "重要保持客户"
    elif x.iloc[0]==0 and x.iloc[1]==1 and x.iloc[2]==0:
        return "一般客户"
    elif x.iloc[0]==0 and x.iloc[1]==0 and x.iloc[2]==1:
        return "重要挽回客户"
    elif x.iloc[0]==0 and x.iloc[1]==0 and x.iloc[2]==0:
        return "流失客户"

In [98]:
#打标签
data["RFM"] = data[["R-SCORE是否大于均值","F-SCORE是否大于均值","M-SCORE是否大于均值"]].apply(RFM,axis=1)
data.sample(10)

,id,R,F,M,R-SCORE,F-SCORE,M-SCORE,R-SCORE是否大于均值,F-SCORE是否大于均值,M-SCORE是否大于均值,RFM
29882,2945371,1,88,1754.0,5,4,3,1,1,1,重要价值客户
34322,2369602,126,18,398.0,1,2,2,0,0,0,流失客户
256,1473361,242,16,298.0,1,2,1,0,0,0,流失客户
10391,2453052,43,1,398.0,2,1,2,0,0,0,流失客户
10176,1962693,2,176,4526.0,5,5,4,1,1,1,重要价值客户
16638,5784593,7,6,378.0,4,1,2,1,0,0,新客户
33628,6427765,3,11,398.0,5,2,2,1,0,0,新客户
5711,2812547,43,34,398.0,2,3,2,0,1,0,一般客户
15699,3363100,4,35,1086.0,5,3,3,1,1,1,重要价值客户
39197,2659388,95,11,398.0,1,2,2,0,0,0,流失客户


In [99]:
#数据汇总
result = data.groupby("RFM").agg({"RFM":"count"})

In [100]:
#数据展示
from pyecharts import options as opts
from pyecharts.charts import TreeMap

data = [
    {"value": int(result['RFM'][0]), "name": result['RFM'].index[0]},
    {"value": int(result['RFM'][1]), "name": result['RFM'].index[1]},
    {"value": int(result['RFM'][2]), "name": result['RFM'].index[2]},
    {"value": int(result['RFM'][3]), "name": result['RFM'].index[3]},
    {"value": int(result['RFM'][4]), "name": result['RFM'].index[4]},
    {"value": int(result['RFM'][5]), "name": result['RFM'].index[5]},
    {"value": int(result['RFM'][6]), "name": result['RFM'].index[6]},
    {"value": int(result['RFM'][7]), "name": result['RFM'].index[7]},
]

c = (
    TreeMap()
    .add("RFM模型", data
    ,label_opts= opts.LabelOpts(font_size= 12,position = "inside"))
    .set_global_opts(title_opts=opts.TitleOpts(title=""))

)   
c.render_notebook()